In [4]:
import os
import shutil
import random
import json
from PIL import Image
import pandas as pd
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

from bertopic import BERTopic

from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance

import sys
sys.path.insert(0, "../../")
sys.path.insert(0, "../../../hindsight/hindsight_server/")
from annotation_helpers import add_hindsight_frame_path, get_entity_image, visualize_annotations, get_entity_image
from annotations_db import HindsightAnnotationsDB

from db import HindsightDB
from utils import make_dir, ocr_results_to_str, add_datetimes

from hindsight_server.config import LLM_MODEL_NAME
from hindsight_server.query.query import load, llm_generate

In [12]:
tweets_df = pd.read_csv("extracted_tweets_1.csv")
tweets_df = add_datetimes(tweets_df)
tweets_df = tweets_df.dropna(subset=['tweet_text'])
tweets_df = tweets_df.drop_duplicates(subset=['tweet_text'], keep="first")

In [ ]:
tweets_df['username'] = tweets_df['username'].apply(lambda x: x.replace("_", " "))

In [5]:
with open("stock_info.json", 'r') as infile:
    stock_d = json.load(infile)

In [9]:
stock_df = pd.DataFrame(stock_d['data']['rows'])

In [27]:
stock_symbols = '|'.join(stock_df['symbol'])
tweets_df['contains_stock_symbol'] = tweets_df['tweet_text'].str.contains(stock_symbols)

In [28]:
tweets_df.loc[tweets_df['contains_stock_symbol']]

,parent_annotations_id,username,user_image,tweet_text,tweet_image_content,image_content_source,retweets,replies,likes,impresssions,...,user_retweeted_identifier,loading_icon,frame_id,timestamp,datetime_utc,datetime_local,5mins_dt,5min_interval,contains_stock_symbol,contains_financial_keyword
0,30,Interesting As Fuck,True,Let him cook,True,NaN,172.1K,→ 380,©65.5K |19.5M A %,©65.5K |19.5M A %,...,NaN,NaN,378016.0,1725994185159,2024-09-10 18:49:45.158999920+00:00,2024-09-10 14:49:45.158999920-04:00,2024-09-10 14:49:45.158999920-04:00,2024-09-10 14:49:28.568000078-04:00,True,False
1,33,Figen,True,"Work smart! Everyone do it in their own style,...",True,From Mónica,171.3K,@ 468,©20.7K Ill 6.3M,©20.7K Ill 6.3M,...,NaN,NaN,378029.0,1725994170649,2024-09-10 18:49:30.648999929+00:00,2024-09-10 14:49:30.648999929-04:00,2024-09-10 14:49:30.648999929-04:00,2024-09-10 14:49:28.568000078-04:00,True,False
3,44,Geiger Capital &,True,This is incredible... CNN's Erin Burnett liter...,True,NaN,179.5K,942,29.3K,Il 9.4M,...,NaN,NaN,378054.0,1725994197640,2024-09-10 18:49:57.640000105+00:00,2024-09-10 14:49:57.640000105-04:00,2024-09-10 14:49:57.640000105-04:00,2024-09-10 14:49:28.568000078-04:00,True,False
4,45,INFOWARS @ @infowars • 5h,True,The American Journal Watch Live: Melania Trump...,True,NaN,176.8K,• 923,© 29.9K | 3.2M,© 29.9K | 3.2M,...,NaN,NaN,378056.0,1725994193436,2024-09-10 18:49:53.436000109+00:00,2024-09-10 14:49:53.436000109-04:00,2024-09-10 14:49:53.436000109-04:00,2024-09-10 14:49:28.568000078-04:00,True,False
5,56,Brett Kollmann,True,The first kicker th arns how to consistently l...,True,NaN,17111,• 83,© 8.9K ||| 3.3M,© 8.9K ||| 3.3M,...,NaN,NaN,378076.0,1725994168568,2024-09-10 18:49:28.568000078+00:00,2024-09-10 14:49:28.568000078-04:00,2024-09-10 14:49:28.568000078-04:00,2024-09-10 14:49:28.568000078-04:00,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,204065,Massimo • @Rainmaker1973 • 22h,True,How SpaceX Starship will take us on Mars,NaN,From Mitul Tripathi,171.2K,@ 307,© 8.9K,I. 1.6M,...,NaN,NaN,434947.0,1727012310025,2024-09-22 13:38:30.025000095+00:00,2024-09-22 09:38:30.025000095-04:00,2024-09-22 09:38:30.025000095-04:00,2024-09-22 09:35:01.256999969-04:00,True,False
9523,204068,Kevin W.,True,This is Morgan. She lost the ability to walk 1...,True,From Laughing Legends,177.5K,© 866,© 214K,Ill 14.4M,...,NaN,NaN,434950.0,1727012246107,2024-09-22 13:37:26.107000113+00:00,2024-09-22 09:37:26.107000113-04:00,2024-09-22 09:37:26.107000113-04:00,2024-09-22 09:35:01.256999969-04:00,True,False
9524,204070,CRAYATOR,True,"18 hours and 23 minutes into a GTA 5, No death...",True,NaN,171.3K,@ 338,48.1K ||| 4.4M,48.1K ||| 4.4M,...,NaN,NaN,434954.0,1727012200484,2024-09-22 13:36:40.483999968+00:00,2024-09-22 09:36:40.483999968-04:00,2024-09-22 09:36:40.483999968-04:00,2024-09-22 09:35:01.256999969-04:00,True,False
9527,204099,Laura,True,THE F1 ACCOUNT PUT A TIMING ON THE LIZARD @GOO,True,NaN,1746.1K,@ 11K,366K || 10.1M,366K || 10.1M,...,NaN,NaN,434994.0,1727012250194,2024-09-22 13:37:30.194000006+00:00,2024-09-22 09:37:30.194000006-04:00,2024-09-22 09:37:30.194000006-04:00,2024-09-22 09:35:01.256999969-04:00,True,False


In [23]:
financial_keywords = [
    'buy', 'sell', 'hold', 'short', 'long', 'invest', 'divest', 'dump',
    'accumulate', 'stock', 'shares', 'equities', 'options', 'market',
    'portfolio', 'bullish', 'bearish', 'undervalued', 'overvalued'
]
pattern_keywords = '|'.join(financial_keywords)
tweets_df['contains_financial_keyword'] = tweets_df['tweet_text'].str.contains(pattern_keywords)

In [24]:
tweets_df.loc[tweets_df['contains_financial_keyword']]

,parent_annotations_id,username,user_image,tweet_text,tweet_image_content,image_content_source,retweets,replies,likes,impresssions,...,user_retweeted_identifier,loading_icon,frame_id,timestamp,datetime_utc,datetime_local,5mins_dt,5min_interval,contains_stock_symbol,contains_financial_keyword
63,440,Elon Musk &,True,"But here's the rub, Tesla never has to bribe K...",NaN,NaN,94.8K 176.6K,94.8K 176.6K,©61.3K | 16M,©61.3K | 16M,...,NaN,NaN,379950.0,1726020432625,2024-09-11 02:07:12.625000+00:00,2024-09-10 22:07:12.625000-04:00,2024-09-10 22:07:12.625000-04:00,2024-09-10 22:05:01.371999979-04:00,False,True
84,636,Elon Musk,True,"But here's the rub, Tesla never has to bribe K...",NaN,NaN,@4.8K 176.6K,@4.8K 176.6K,© 61.3K ||| 16M,© 61.3K ||| 16M,...,NaN,NaN,380280.0,1726020426539,2024-09-11 02:07:06.539000034+00:00,2024-09-10 22:07:06.539000034-04:00,2024-09-10 22:07:06.539000034-04:00,2024-09-10 22:05:01.371999979-04:00,False,True
96,730,James Woods,True,Twenty-three years ago today I picked up the p...,True,NaN,173.8K,• 641,© 30.7K Ill 2.7M,© 30.7K Ill 2.7M,...,NaN,NaN,383033.0,1726101333044,2024-09-12 00:35:33.043999910+00:00,2024-09-11 20:35:33.043999910-04:00,2024-09-11 20:35:33.043999910-04:00,2024-09-11 20:35:00.017999887-04:00,False,True
131,944,kira '#,True,and they said money doesn't buy happiness,True,From k,2770,• 192,∞ 1.6K,Ill 84K,...,NaN,NaN,383250.0,1726106838833,2024-09-12 02:07:18.832999945+00:00,2024-09-11 22:07:18.832999945-04:00,2024-09-11 22:07:18.832999945-04:00,2024-09-11 22:06:29.461999893-04:00,False,True
170,1278,David Santa Carla,True,This kid has a TikTok channel where he tries t...,True,NaN,@2.4K 1710.6K,@2.4K 1710.6K,© 83K,I 9.9M,...,NaN,NaN,384616.0,1726147629521,2024-09-12 13:27:09.520999907+00:00,2024-09-12 09:27:09.520999907-04:00,2024-09-12 09:27:09.520999907-04:00,2024-09-12 09:25:32.526999950-04:00,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9400,202949,Jv Shah,True,Think twice before buying! Here are 10 gadgets...,True,NaN,17109,62,393,Il. 202K,...,NaN,NaN,430172.0,1726884431554,2024-09-21 02:07:11.553999901+00:00,2024-09-20 22:07:11.553999901-04:00,2024-09-20 22:07:11.553999901-04:00,2024-09-20 22:07:11.553999901-04:00,False,True
9426,203170,DC Draino,True,If Trump temporarily caps credit card interest...,True,NaN,174.2K ©15.2K | 348K,• 452,174.2K ©15.2K | 348K,174.2K ©15.2K | 348K,...,NaN,NaN,430743.0,1726891766640,2024-09-21 04:09:26.640000105+00:00,2024-09-21 00:09:26.640000105-04:00,2024-09-21 00:09:26.640000105-04:00,2024-09-21 00:07:21.338000059-04:00,False,True
9431,203230,Vivek Ramaswamy,True,"When I was a kid, it felt like the last embers...",True,NaN,921K 7111k @64.8K 1.7M,921K 7111k @64.8K 1.7M,921K 7111k @64.8K 1.7M,921K 7111k @64.8K 1.7M,...,NaN,NaN,430757.0,1726891768712,2024-09-21 04:09:28.711999893+00:00,2024-09-21 00:09:28.711999893-04:00,2024-09-21 00:09:28.711999893-04:00,2024-09-21 00:07:21.338000059-04:00,False,True
9507,203951,Collin Rugg,True,"Squatters have taken over a nearly 10,000 squa...",True,NaN,@4.4K 179.4K,@4.4K 179.4K,©30.5K Ill 9.3M,©30.5K Ill 9.3M,...,NaN,NaN,434867.0,1727012107537,2024-09-22 13:35:07.536999941+00:00,2024-09-22 09:35:07.536999941-04:00,2024-09-22 09:35:07.536999941-04:00,2024-09-22 09:35:01.256999969-04:00,False,True


In [17]:
tweets_df['contains_stock_symbol'] = tweets_df['tweet_text'].str.contains(r'\$\b[A-Z]{1,5}\b')


In [19]:
tweets_df.loc[tweets_df['contains_stock_symbol']]

,parent_annotations_id,username,user_image,tweet_text,tweet_image_content,image_content_source,retweets,replies,likes,impresssions,...,more_posted,user_retweeted_identifier,loading_icon,frame_id,timestamp,datetime_utc,datetime_local,5mins_dt,5min_interval,contains_stock_symbol
403,134092,Le Shrub,True,"$GME will be below $20 by June 21st. If wrong,...",NaN,NaN,174,@ 20,161,1l 10.6K,...,NaN,NaN,NaN,24850.0,1717429951306,2024-06-03 15:52:31.305999994+00:00,2024-06-03 11:52:31.305999994-04:00,2024-06-03 11:52:31.305999994-04:00,2024-06-03 11:51:19.397000074-04:00,True
514,3786,The Transcript,True,@TheTranscript_• 15h $ORCL CTO Larry Ellison: ...,True,NaN,17448,@ 41,2.1K,1l 571K,...,NaN,NaN,NaN,397780.0,1726342941179,2024-09-14 19:42:21.178999901+00:00,2024-09-14 15:42:21.178999901-04:00,2024-09-14 15:42:21.178999901-04:00,2024-09-14 15:41:45.871000051-04:00,True
620,4217,The Transcript,True,@TheTranscript_• 15h $ORCL CTO Larry Ellison: ...,True,NaN,17448,@ 41,2.1K,1l 571K,...,NaN,NaN,NaN,398071.0,1726342930987,2024-09-14 19:42:10.986999989+00:00,2024-09-14 15:42:10.986999989-04:00,2024-09-14 15:42:10.986999989-04:00,2024-09-14 15:41:45.871000051-04:00,True
1316,139113,Evan,True,"Nvidia $NVDA CEO Jensen Huang just said: ""Well...",NaN,NaN,1777,@20,© 747,Ill 118K,...,NaN,NaN,NaN,38431.0,1716436545766,2024-05-23 03:55:45.766000032+00:00,2024-05-22 23:55:45.766000032-04:00,2024-05-22 23:55:45.766000032-04:00,2024-05-22 23:55:02.732000113-04:00,True
1320,8066,The Vigilant Fox,True,NEW: Joe Rogan Tells Adam Sandler That Will Sm...,True,NaN,17588,@ 353,7.8K,NaN,...,NaN,NaN,NaN,293333.0,1723665317368,2024-08-14 19:55:17.368000031+00:00,2024-08-14 15:55:17.368000031-04:00,2024-08-14 15:55:17.368000031-04:00,2024-08-14 15:55:06.948999882-04:00,True
1599,140441,Whole Mars Catalog,True,Tesla Semi early fleet seeing uptime greater t...,NaN,NaN,1719,@ 12,467,Ill 430K,...,NaN,NaN,NaN,41714.0,1716306498467,2024-05-21 15:48:18.467000008+00:00,2024-05-21 11:48:18.467000008-04:00,2024-05-21 11:48:18.467000008-04:00,2024-05-21 11:45:36.826999903-04:00,True
2374,143676,unusual whales,True,"BREAKING: BlackRock, $BLK, and Citadel Securit...",NaN,NaN,172.8K,@11K,©14.5K | 2.7M,©14.5K | 2.7M,...,NaN,NaN,NaN,58559.0,1717632972306,2024-06-06 00:16:12.305999994+00:00,2024-06-05 20:16:12.305999994-04:00,2024-06-05 20:16:12.305999994-04:00,2024-06-05 20:15:16.344000101-04:00,True
2689,145212,Dog,True,You are not ready! $DOG,True,NaN,2767,© 24,700,Il. 16.9M,...,NaN,NaN,NaN,68929.0,1717854288558,2024-06-08 13:44:48.558000088+00:00,2024-06-08 09:44:48.558000088-04:00,2024-06-08 09:44:48.558000088-04:00,2024-06-08 09:41:48.230999947-04:00,True
3066,146942,unusual_whales,True,"This was Andrew Left of Citron Research, who s...",True,NaN,17292,• 424,• 2.5K,I 727K,...,NaN,NaN,NaN,73989.0,1718026098120,2024-06-10 13:28:18.119999886+00:00,2024-06-10 09:28:18.119999886-04:00,2024-06-10 09:28:18.119999886-04:00,2024-06-10 09:25:00.802000046-04:00,True
3320,148461,Le Shrub,True,"Rumor that a hedge fund pod blew up, which is ...",NaN,NaN,1738,@45,458,Ill 156K,...,NaN,NaN,NaN,81786.0,1718161386221,2024-06-12 03:03:06.220999956+00:00,2024-06-11 23:03:06.220999956-04:00,2024-06-11 23:03:06.220999956-04:00,2024-06-11 23:00:29.765000105-04:00,True


In [22]:
tweets_df.loc[tweets_df['username'] == "Le Shrub"]

,parent_annotations_id,username,user_image,tweet_text,tweet_image_content,image_content_source,retweets,replies,likes,impresssions,...,more_posted,user_retweeted_identifier,loading_icon,frame_id,timestamp,datetime_utc,datetime_local,5mins_dt,5min_interval,contains_stock_symbol
403,134092,Le Shrub,True,"$GME will be below $20 by June 21st. If wrong,...",NaN,NaN,174,@ 20,161,1l 10.6K,...,NaN,NaN,NaN,24850.0,1717429951306,2024-06-03 15:52:31.305999994+00:00,2024-06-03 11:52:31.305999994-04:00,2024-06-03 11:52:31.305999994-04:00,2024-06-03 11:51:19.397000074-04:00,True
586,135119,Le Shrub,True,Great news!,NaN,NaN,17,NaN,• 12,Ill 3.3K,...,NaN,NaN,NaN,29545.0,1717124230500,2024-05-31 02:57:10.500000+00:00,2024-05-30 22:57:10.500000-04:00,2024-05-30 22:57:10.500000-04:00,2024-05-30 22:55:02.559999943-04:00,False
1940,141826,Le Shrub,True,"finally someone said it, thank you xxx",NaN,NaN,27,NaN,14,1l 2.9K,...,NaN,NaN,NaN,51389.0,1717463317338,2024-06-04 01:08:37.338000059+00:00,2024-06-03 21:08:37.338000059-04:00,2024-06-03 21:08:37.338000059-04:00,2024-06-03 21:07:54.441999912-04:00,False
2387,143752,Le Shrub,True,Replying to @Edark94 Energy and food security....,NaN,NaN,17,8,17,Ill 3.5K,...,NaN,NaN,NaN,59227.0,1717644863379,2024-06-06 03:34:23.378999949+00:00,2024-06-05 23:34:23.378999949-04:00,2024-06-05 23:34:23.378999949-04:00,2024-06-05 23:32:29.555999994-04:00,False
2677,145070,Le Shrub,True,"""If it looks like a Pump, and it walks like a ...",NaN,NaN,277,@ 13,191,Ill 16.1K,...,NaN,NaN,NaN,68532.0,1717862543401,2024-06-08 16:02:23.401000023+00:00,2024-06-08 12:02:23.401000023-04:00,2024-06-08 12:02:23.401000023-04:00,2024-06-08 12:00:03.892999887-04:00,False
3046,146822,Le Shrub,True,I don't get the strategy,NaN,NaN,171,• 16,© 53,1 20.2K W %,...,NaN,NaN,NaN,73925.0,1718026153307,2024-06-10 13:29:13.306999922+00:00,2024-06-10 09:29:13.306999922-04:00,2024-06-10 09:29:13.306999922-04:00,2024-06-10 09:25:00.802000046-04:00,False
3320,148461,Le Shrub,True,"Rumor that a hedge fund pod blew up, which is ...",NaN,NaN,1738,@45,458,Ill 156K,...,NaN,NaN,NaN,81786.0,1718161386221,2024-06-12 03:03:06.220999956+00:00,2024-06-11 23:03:06.220999956-04:00,2024-06-11 23:03:06.220999956-04:00,2024-06-11 23:00:29.765000105-04:00,True
3372,148606,Le Shrub,True,"Rumor that a hedge fund pod blew up, which is ...",NaN,NaN,1738,• 45,458,Ill 156K,...,NaN,NaN,NaN,81928.0,1718161392433,2024-06-12 03:03:12.433000088+00:00,2024-06-11 23:03:12.433000088-04:00,2024-06-11 23:03:12.433000088-04:00,2024-06-11 23:00:29.765000105-04:00,True
3429,148830,Le Shrub,True,Why would I be afraid of a made-up number?,NaN,NaN,171,NaN,025,1l 4.4K,...,NaN,NaN,NaN,82784.0,1718214850818,2024-06-12 17:54:10.818000078+00:00,2024-06-12 13:54:10.818000078-04:00,2024-06-12 13:54:10.818000078-04:00,2024-06-12 13:50:49.799999952-04:00,False
3509,149193,Le Shrub,True,Why would I be afraid of a made-up number? 025,NaN,NaN,171,NaN,025,Ill 4.4K,...,NaN,NaN,NaN,83512.0,1718214846629,2024-06-12 17:54:06.628999949+00:00,2024-06-12 13:54:06.628999949-04:00,2024-06-12 13:54:06.628999949-04:00,2024-06-12 13:50:49.799999952-04:00,False


In [15]:
stock_df.loc[stock_df['symbol'] == "NVDA"]

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
4528,NVDA,NVIDIA Corporation Common Stock,$116.26,0.26,0.224%,205795239,2851857800000.00,United States,1999,Semiconductors,Technology,/market-activity/stocks/nvda


In [13]:
tweets_df.loc[tweets_df['tweet_text'].str.contains("NVDA")]

,parent_annotations_id,username,user_image,tweet_text,tweet_image_content,image_content_source,retweets,replies,likes,impresssions,...,community_notes,more_posted,user_retweeted_identifier,loading_icon,frame_id,timestamp,datetime_utc,datetime_local,5mins_dt,5min_interval
1316,139113,Evan,True,"Nvidia $NVDA CEO Jensen Huang just said: ""Well...",NaN,NaN,1777,@20,© 747,Ill 118K,...,NaN,NaN,NaN,NaN,38431.0,1716436545766,2024-05-23 03:55:45.766000032+00:00,2024-05-22 23:55:45.766000032-04:00,2024-05-22 23:55:45.766000032-04:00,2024-05-22 23:55:02.732000113-04:00
3320,148461,Le Shrub,True,"Rumor that a hedge fund pod blew up, which is ...",NaN,NaN,1738,@45,458,Ill 156K,...,NaN,NaN,NaN,NaN,81786.0,1718161386221,2024-06-12 03:03:06.220999956+00:00,2024-06-11 23:03:06.220999956-04:00,2024-06-11 23:03:06.220999956-04:00,2024-06-11 23:00:29.765000105-04:00
3372,148606,Le Shrub,True,"Rumor that a hedge fund pod blew up, which is ...",NaN,NaN,1738,• 45,458,Ill 156K,...,NaN,NaN,NaN,NaN,81928.0,1718161392433,2024-06-12 03:03:12.433000088+00:00,2024-06-11 23:03:12.433000088-04:00,2024-06-11 23:03:12.433000088-04:00,2024-06-11 23:00:29.765000105-04:00
5042,158857,NaN,True,$NVDA is down 20bps and the FED is sitting the...,NaN,NaN,174,© 16,206,Ill 14.1K,...,NaN,NaN,NaN,NaN,100601.0,1718639167303,2024-06-17 15:46:07.302999973+00:00,2024-06-17 11:46:07.302999973-04:00,2024-06-17 11:46:07.302999973-04:00,2024-06-17 11:45:18.407999992-04:00


In [14]:
tweets_df.loc[tweets_df['tweet_text'].str.contains("Nvidia")]

,parent_annotations_id,username,user_image,tweet_text,tweet_image_content,image_content_source,retweets,replies,likes,impresssions,...,community_notes,more_posted,user_retweeted_identifier,loading_icon,frame_id,timestamp,datetime_utc,datetime_local,5mins_dt,5min_interval
483,3672,Autism Capital,True,"@AutismCapi... • 22h PETER THIEL: ""Making mone...",True,NaN,1772,• 30,839,Ill 113K,...,NaN,NaN,NaN,NaN,397705.0,1726343061722,2024-09-14 19:44:21.721999884+00:00,2024-09-14 15:44:21.721999884-04:00,2024-09-14 15:44:21.721999884-04:00,2024-09-14 15:41:45.871000051-04:00
687,135597,Ben Lang,True,Nvidia employees share their stock compensatio...,NaN,NaN,1784,© 10,833,Ill 478K,...,NaN,NaN,NaN,NaN,31998.0,1716910396450,2024-05-28 15:33:16.450000048+00:00,2024-05-28 11:33:16.450000048-04:00,2024-05-28 11:33:16.450000048-04:00,2024-05-28 11:31:01.142999887-04:00
1316,139113,Evan,True,"Nvidia $NVDA CEO Jensen Huang just said: ""Well...",NaN,NaN,1777,@20,© 747,Ill 118K,...,NaN,NaN,NaN,NaN,38431.0,1716436545766,2024-05-23 03:55:45.766000032+00:00,2024-05-22 23:55:45.766000032-04:00,2024-05-22 23:55:45.766000032-04:00,2024-05-22 23:55:02.732000113-04:00
2438,143882,greg,True,The market cap of Nvidia is up over $200 billi...,NaN,NaN,17743,@ 288,8.6K,I 1M,...,NaN,NaN,NaN,NaN,59573.0,1717644848925,2024-06-06 03:34:08.924999952+00:00,2024-06-05 23:34:08.924999952-04:00,2024-06-05 23:34:08.924999952-04:00,2024-06-05 23:32:29.555999994-04:00
2492,144125,greg,True,The market cap of Nvidia is up over $200 billi...,NaN,NaN,17743,• 288,© 8.6K,I| 1M,...,NaN,NaN,NaN,NaN,60145.0,1717644853100,2024-06-06 03:34:13.099999905+00:00,2024-06-05 23:34:13.099999905-04:00,2024-06-05 23:34:13.099999905-04:00,2024-06-05 23:32:29.555999994-04:00
3383,17594,Sean Kelly &,True,The way that Jensen Huang runs Nvidia is wild:...,True,NaN,1711K,136,© 9.7K,I 2.1M,...,NaN,NaN,NaN,NaN,347599.0,1725077777634,2024-08-31 04:16:17.634000063+00:00,2024-08-31 00:16:17.634000063-04:00,2024-08-31 00:16:17.634000063-04:00,2024-08-31 00:15:01.312999964-04:00
3496,18060,Sean Kelly,True,The way that Jen: ^ 'uang runs Nvidia is wild:...,True,NaN,1711K,136,© 9.7K,I 2.1M,...,NaN,NaN,NaN,NaN,348984.0,1725107024524,2024-08-31 12:23:44.523999929+00:00,2024-08-31 08:23:44.523999929-04:00,2024-08-31 08:23:44.523999929-04:00,2024-08-31 08:23:44.523999929-04:00
5319,161644,Ben Lang,True,If you joined Nvidia 5 years ago as a mid-leve...,NaN,NaN,17631,73,© 8.9K,Ill 1.6M,...,NaN,NaN,NaN,NaN,114389.0,1718845984036,2024-06-20 01:13:04.036000013+00:00,2024-06-19 21:13:04.036000013-04:00,2024-06-19 21:13:04.036000013-04:00,2024-06-19 21:10:04.036000013-04:00
7843,186291,Elon Musk,True,"Nice work by @xAI team, @X team, @Nvidia & sup...",NaN,NaN,177.8K,@ 3.7K,© 70.7K || 15.5M,© 70.7K || 15.5M,...,NaN,NaN,NaN,NaN,220122.0,1721740686769,2024-07-23 13:18:06.769000053+00:00,2024-07-23 09:18:06.769000053-04:00,2024-07-23 09:18:06.769000053-04:00,2024-07-23 09:16:44.733999968-04:00
